CNN Model to detect horse and human

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from zipfile import ZipFile
with ZipFile('archive.zip') as zip:
    zip.extractall()

In [ ]:
# tf.data.Dataset

In [7]:
TRAIN_DATA = 'horse-or-human/train'

Load Dataset using tensorflow

In [11]:
train_data = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DATA, # it must contain two(or more) sub directory that contains images for each class
    image_size=(300,300), # size of image
    batch_size= 128,
    label_mode='binary'
)

Found 1027 files belonging to 2 classes.


In [12]:
recycle_layer = tf.keras.layers.Rescaling(scale= 1. / 255)
train_data_scaled = train_data.map(lambda image,label:( recycle_layer(image),label))

In [18]:
train_data_final = (train_data_scaled
                    .cache()
                    .shuffle(buffer_size=1000)
                    .prefetch(buffer_size=tf.data.AUTOTUNE))